In [1]:
import random
from collections import Counter


class ModelStatus:
    queueLength = 0
    isThreadOneBlocked = False
    isThreadTwoBlocked = False

    def __init__(self, queueLength, isThreadOneBlocked, isThreadTwoBlocked):
        self.queueLength = queueLength
        self.isThreadOneBlocked = isThreadOneBlocked
        self.isThreadTwoBlocked = isThreadTwoBlocked

    def to_string(self):
        return self.queueLength, 1 if self.isThreadOneBlocked else 0, 1 if self.isThreadTwoBlocked else 0


class PseudoRandom:

    a = 134279
    Rn = 1
    m = 313107

    def random(self):
        self.Rn = self.a * self.Rn % self.m
        return self.Rn / self.m


pseudo_random = PseudoRandom()

statusHistory = []
handle_request = []

p = 0.1
pi_1 = 0.7
pi_2 = 0.5

maxQueueLength = 2

isNewRequest = False
queueLength = 0
isThreadOneBlocked = False
isThreadTwoBlocked = False


def generate_state(probability):
    return pseudo_random.random() < probability


def iterate():
    global isNewRequest
    global queueLength
    global isThreadOneBlocked
    global isThreadTwoBlocked

    handling_requests_calculater = 0

    isNewRequest = not generate_state(p)
    if isThreadOneBlocked:
        isThreadOneBlocked = generate_state(pi_1)
        if not isThreadOneBlocked:
            handling_requests_calculater += 1

    if isThreadTwoBlocked:
        isThreadTwoBlocked = generate_state(pi_2)
        if not isThreadTwoBlocked:
            handling_requests_calculater += 1

    if isNewRequest and queueLength < maxQueueLength:
        queueLength += 1

    if not isThreadOneBlocked and queueLength > 0:
        isThreadOneBlocked = True
        queueLength -= 1

    if not isThreadTwoBlocked and queueLength > 0:
        isThreadTwoBlocked = True
        queueLength -= 1
---
    statusHistory.append(ModelStatus(queueLength, isThreadOneBlocked, isThreadTwoBlocked).to_string())
    handle_request.append(handling_requests_calculater)

iterations = 1000000
for i in range(iterations):
    iterate()


counter = Counter(statusHistory)

for state in counter:
    print(state, round(counter[state]/iterations * 100, 2),'%')


L__q = sum(list(map(lambda state: state[0] * counter[state], counter))) / iterations
print("\nL__q:", L__q)

A = sum(handle_request) / iterations
print("A:", A)

W__q = L__q / A
print("W__q: ", W__q)

K__1 = sum(list(map(lambda state: state[1] * counter[state], counter))) / iterations
print("K__1:", K__1)
K__2 = sum(list(map(lambda state: state[2] * counter[state], counter))) / iterations
print("K__2:", K__2)

(0, 1, 0) 11.18 %
(0, 1, 1) 32.74 %
(1, 1, 1) 36.83 %
(2, 1, 1) 17.79 %
(0, 0, 1) 0.53 %
(0, 0, 0) 0.94 %

L__q: 0.724026
A: 0.739051
W__q:  0.979669873932922
K__1: 0.985262
K__2: 0.878817
